# Python unittest framework

To run tests in the notebook environment, we need a utility method:

In [3]:
def run_tests(test_cls):
    tests = unittest.TestLoader().loadTestsFromTestCase(test_cls)
    unittest.TextTestRunner(verbosity=2).run(tests)

## unittest.TestCase

In [4]:
import unittest

class MyTest(unittest.TestCase):

    def test_sanity(self):
        """Verify sanity.
        """
        self.assertTrue(True)

run_tests(MyTest)

test_sanity (__main__.MyTest)
Verify sanity. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


## Recipe

In [5]:
class Recipe(object):
    """For now, just an encapsulation of ingredients.
    """
    def __init__(self, ingredients=None):
        """
        :type ingredients: [:py:class:`Ingredient`, ...]
        :arg ingredients: list of recipe ingredients
        """
        self.ingredients = ingredients

## RecipeTests 

In [57]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    def test_init(self):
        """Verify member data initialization.
        """
        test_ingredients = [Ingredient(), Ingredient()]
        test_recipe = Recipe(ingredients=test_ingredients)
        self.assertEqual(test_ingredients, test_recipe.ingredients)
        
run_tests(RecipeTests)

test_init (__main__.RecipeTests)
Verify member data initialization. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## No Ingredient class? No problem. Introducing Mock.

In [52]:
import mock

class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    def test_init(self):
        """Verify member data initialization.
        """
        test_ingredients = mock.Mock(name='ingredients')
        test_recipe = Recipe(ingredients=test_ingredients)
        self.assertEqual(test_ingredients, test_recipe.ingredients)
        
run_tests(RecipeTests)

test_init (__main__.RecipeTests)
Verify member data initialization. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


## Recipe.is_vegetarian

In [22]:
class Recipe(object):
    """For now, just an encapsulation of ingredients.
    """
    def __init__(self, ingredients=None):
        self.ingredients = ingredients
        
    def is_vegetarian(self):
        """Recipe is vegetarian iff all ingredients are vegetarian.
        
        :rtype: bool
        :return: True if recipe is vegetarian, False otherwise
        
        """
        if not self.ingredients:
            raise ValueError("Recipe has no ingredients")
        for ingredient in self.ingredients:
            if not ingredient.is_vegetarian():
                return False
        return True

## RecipeTests.test_is_vegetarian

In [81]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    def test_is_vegetarian(self):
        """Verify calls made by is_vegetarian.
        """
        test_ingredients = mock.Mock(name='ingredients')
        test_recipe = Recipe(ingredients=test_ingredients)
        test_recipe.is_vegetarian()
        
run_tests(RecipeTests)

test_is_vegetarian (__main__.RecipeTests)
Verify calls made by is_vegetarian. ... ERROR

ERROR: test_is_vegetarian (__main__.RecipeTests)
Verify calls made by is_vegetarian.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-81-1ced4e2db9f0>", line 9, in test_is_vegetarian
    test_recipe.is_vegetarian()
  File "<ipython-input-22-77be25c9c2ac>", line 16, in is_vegetarian
    for ingredient in self.ingredients:
TypeError: 'Mock' object is not iterable

----------------------------------------------------------------------
Ran 1 test in 0.008s

FAILED (errors=1)


## Mock object not iterable? No problem. Introducing MagicMock.

**Try it:** Replace Mock with MagicMock.

**Warning:** Be careful with magic.

## Ingredient class

In [66]:
class Ingredient(object):
    """Not very interesting yet. 
    """     
    def is_vegetarian(self):
        raise NotImplementedError()

**Try it:** Re-run original RecipeTests.test_init, now that Ingredient is defined.

## RecipeTests.test_is_vegetarian with Ingredient objects

In [68]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    def test_is_vegetarian(self):
        """Verify calls made by is_vegetarian.
        """
        test_ingredients = [Ingredient(), Ingredient()]
        test_recipe = Recipe(ingredients=test_ingredients)
        test_recipe.is_vegetarian()
        
run_tests(RecipeTests)

test_is_vegetarian (__main__.RecipeTests)
Verify calls made by is_vegetarian. ... ERROR

ERROR: test_is_vegetarian (__main__.RecipeTests)
Verify calls made by is_vegetarian.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-68-7741a9de72ae>", line 9, in test_is_vegetarian
    test_recipe.is_vegetarian()
  File "<ipython-input-22-77be25c9c2ac>", line 17, in is_vegetarian
    if not ingredient.is_vegetarian():
  File "<ipython-input-66-9d468edebd58>", line 5, in is_vegetarian
    raise NotImplementedError()
NotImplementedError

----------------------------------------------------------------------
Ran 1 test in 0.005s

FAILED (errors=1)


## Not ready to implement Ingredient.is_vegetarian? No problem. Introducing mock.patch.

In [82]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    @mock.patch.object(Ingredient, 'is_vegetarian')
    def test_is_vegetarian(self, is_ingredient_vegetarian):
        """Verify calls made by is_vegetarian.
        """
        # Set up test data
        test_ingredients = [Ingredient(), Ingredient()]
        test_recipe = Recipe(ingredients=test_ingredients)
        
        # Make call
        test_recipe.is_vegetarian()
        
        # Verify mocks
        self.assertEqual(2, is_ingredient_vegetarian.call_count)
        
run_tests(RecipeTests)

test_is_vegetarian (__main__.RecipeTests)
Verify calls made by is_vegetarian. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


## Mock.return_value and Mock.call_count

In [103]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    @mock.patch.object(Ingredient, 'is_vegetarian')
    def test_is_vegetarian__ingredients_vegetarian(self, is_veg_ingredient):
        """Verify return value of is_vegetarian when ingredients are vegetarian.
        """
        # Set up mocks and test data 
        is_veg_ingredient.return_value = True
        test_ingredients = [Ingredient(), Ingredient()]
        test_recipe = Recipe(ingredients=test_ingredients)
        
        # Make call
        result = test_recipe.is_vegetarian()
        
        # Verify result
        self.assertTrue(result)
        
        # Verify mocks
        self.assertEqual(len(test_ingredients), is_veg_ingredient.call_count)
        
    @mock.patch.object(Ingredient, 'is_vegetarian')
    def test_is_vegetarian__ingredients_non_vegetarian(self, is_veg_ingredient):
        """Verify return value of is_vegetarian when ingredients are not vegetarian.
        """
        pass
        
run_tests(RecipeTests)

test_is_vegetarian__ingredients_non_vegetarian (__main__.RecipeTests)
Verify return value of is_vegetarian when ingredients are not vegetarian. ... ok
test_is_vegetarian__ingredients_vegetarian (__main__.RecipeTests)
Verify return value of is_vegetarian when ingredients are vegetarian. ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.008s

OK


**Try it:** Implement test_is_vegetarian__ingredients_non_vegetarian.

## Mock.side_effect

In [106]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    @mock.patch.object(Ingredient, 'is_vegetarian')
    def test_is_vegetarian__some_ingredients_vegetarian(self, is_veg_ingredient):
        """Verify return value of is_vegetarian when ingredients are vegetarian.
        """
        # Set up mocks and test data
        is_veg_ingredient.side_effect = [True, False, True]
        test_ingredients = [Ingredient(), Ingredient(), Ingredient()]
        test_recipe = Recipe(ingredients=test_ingredients)
        
        # Make call
        result = test_recipe.is_vegetarian()
        
        # Verify result
        self.assertFalse(result)
        
        # Verify mocks
        # Stop calling ingredient.is_vegetarian after first False return value
        self.assertEqual(2, is_veg_ingredient.call_count)
        
run_tests(RecipeTests)

test_is_vegetarian__some_ingredients_vegetarian (__main__.RecipeTests)
Verify return value of is_vegetarian when ingredients are vegetarian. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


## Recipe and Ingredient is_vegetarian methods support _strict_ keyword argument

In [110]:
class Recipe(object):
    """For now, just an encapsulation of ingredients.
    """
    def __init__(self, ingredients=None):
        self.ingredients = ingredients
        
    def is_vegetarian(self, strict=True):
        """Recipe is vegetarian iff all ingredients are vegetarian.
        
        :rtype: bool
        :return: True if recipe is vegetarian, False otherwise
        
        :type strict: bool
        :arg strict: True if only strictly vegetarian recipes should pass
        
        """
        if not self.ingredients:
            raise ValueError("Recipe has no ingredients")
        for ingredient in self.ingredients:
            if not ingredient.is_vegetarian(strict=strict):
                return False
        return True

class Ingredient(object):
    """Not very interesting yet. 
    """     
    def is_vegetarian(self, strict=True):
        raise NotImplementedError()

## Mock.assert_called_once_with

In [111]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    @mock.patch.object(Ingredient, 'is_vegetarian')
    def test_is_vegetarian__strict__one_ingredient(self, is_veg_ingredient):
        """Verify calls made by is_vegetarian.
        """
        # Set up mocks and test data
        is_veg_ingredient.return_value = True
        mock_strict_arg = mock.Mock(name='strict_arg')
        test_recipe = Recipe(ingredients=[Ingredient()])

        # Make call
        test_recipe.is_vegetarian(strict=mock_strict_arg)
        
        # Verify mocks
        is_veg_ingredient.assert_called_once_with(strict=mock_strict_arg)
       
run_tests(RecipeTests)

test_is_vegetarian__strict (__main__.RecipeTests)
Verify calls made by is_vegetarian. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


## Mock.call_args_list and Mock.call

In [101]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    @mock.patch.object(Ingredient, 'is_vegetarian')
    def test_is_vegetarian__strict__multiple_ingredients(self, is_veg_ingredient):
        """Verify calls made by is_vegetarian.
        """
        # Set up mocks and test data
        is_veg_ingredient.return_value = True
        mock_strict_arg = mock.Mock(name='strict_arg')
        test_ingredients = [Ingredient(), Ingredient()]
        test_recipe = Recipe(ingredients=test_ingredients)

        # Make call
        test_recipe.is_vegetarian(strict=mock_strict_arg)
        
        # Verify mocks
        mock_calls = is_veg_ingredient.call_args_list
        self.assertEqual(len(test_ingredients), len(mock_calls))
        for mock_call in mock_calls:
            self.assertEqual(mock.call(strict=mock_strict_arg), mock_call)
        
run_tests(RecipeTests)

test_is_vegetarian__strict (__main__.RecipeTests)
Verify calls made by is_vegetarian. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


## unittest.TestCase.assertRaisesRegexp

In [ ]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    def test_is_vegetarian__no_ingredients(self):
        """Verify ValueError if no ingredients are set on recipe.
        """
        # Set up mocks and test data
        for no_ingredients in (None, []):
            test_recipe = Recipe(ingredients=no_ingredients)

            # Make call
            self.assertRaisesRegexp(ValueError,
                                   'foo',
                                    test_recipe.is_vegetarian)
        
        # Verify mocks
        mock_calls = is_veg_ingredient.call_args_list
        self.assertEqual(len(test_ingredients), len(mock_calls))
        for mock_call in mock_calls:
            self.assertEqual(mock.call(strict=mock_strict_arg), mock_call)
        
run_tests(RecipeTests)